### For standalone testing

In [84]:
#%run .././environment/env1.ipynb
#parm_AHRQCountySDOH_years = ['2015', '2016', '2017']
#parm_AHRQCountySDOH_surveys = ["ACS", "AHA", "AMFAR", "CAF", "CCBP", "CDCSVI", "CEN", "CRDC", "EPAA", "FARA", "FEA", "HHC", "HIFLD", "HRSA", "MHSVI", "MP", "NCHS", "NEPHTN", "NHC", "NOAAS", "POS", "SAHIE", "SAIPE", "SEDA"]
#parm_AHRQCountySDOH_questions = ["CDCW_INJURY_DTH_RATE", "CDCW_TRANSPORT_DTH_RATE", "CDCW_SELFHARM_DTH_RATE", "CDCW_ASSAULT_DTH_RATE", "CHR_TOT_MENTAL_PROV", "CHR_MENTAL_PROV_RATE", "CHR_SEGREG_BLACK", "CHR_PCT_ALCOHOL_DRIV_DEATH", "CHR_PCT_EXCESS_DRINK", "CHR_PCT_FOOD", "CHR_SEGREG_BLACK", "CHR_SEGREG_NON_WHITE"]

### Extract data

In [85]:
AHRQCountySDOH_year = parm_AHRQCountySDOH_years.pop(0)

# Import the first excel file and create dataframe
url = 'https://www.ahrq.gov/sites/default/files/wysiwyg/sdoh/SDOH_'+  AHRQCountySDOH_year +'_COUNTY_1_0.xlsx'
r = requests.get(url)
with open('./ahrq.xlsx', 'wb') as f:
    f.write(r.content) 
dfAHRQCountySDOH = pd.read_excel('./ahrq.xlsx', sheet_name = 'Data')
dfAHRQCountySDOH = dfAHRQCountySDOH.drop(dfAHRQCountySDOH.columns[[ 0,1,2,5,6 ]],axis = 1)
dfAHRQCountySDOH['YEAR'] =  AHRQCountySDOH_year

# Import the remaining excel files and append to dataframe
for AHRQCountySDOH_year in parm_AHRQCountySDOH_years:
    url = 'https://www.ahrq.gov/sites/default/files/wysiwyg/sdoh/SDOH_'+  AHRQCountySDOH_year +'_COUNTY_1_0.xlsx'
    r = requests.get(url)
    with open('./ahrq.xlsx', 'wb') as f:
        f.write(r.content) 
    dfAHRQCountySDOHnext = pd.read_excel('./ahrq.xlsx', sheet_name = 'Data')
    dfAHRQCountySDOHnext = dfAHRQCountySDOHnext.drop(dfAHRQCountySDOHnext.columns[[ 0,1,2,5,6 ]],axis = 1)
    dfAHRQCountySDOHnext['YEAR'] =  AHRQCountySDOH_year
    dfAHRQCountySDOH = dfAHRQCountySDOH.append(dfAHRQCountySDOHnext)

### Clean data
Convert to numeric and remove county from names.

In [86]:
dfAHRQCountySDOH.iloc[:, 3:] = dfAHRQCountySDOH.iloc[:, 3:].apply(pd.to_numeric, errors= 'ignore')
dfAHRQCountySDOH['COUNTY'] = dfAHRQCountySDOH['COUNTY'].str.replace(' County','')

Only include surveys and questions defined in parameters.

In [87]:
dfAHRQCountySDOHred = dfAHRQCountySDOH[["STATE", "COUNTY", "YEAR"]] 

dfAHRQCountySDOHsel = dfAHRQCountySDOH[dfAHRQCountySDOH.columns[pd.Series(dfAHRQCountySDOH.columns).str.startswith(tuple(parm_AHRQCountySDOH_surveys))]]
dfAHRQCountySDOHred2 = pd.concat([dfAHRQCountySDOHred, dfAHRQCountySDOHsel], axis=1)

dfAHRQCountySDOHsel = dfAHRQCountySDOH[parm_AHRQCountySDOH_questions]
dfAHRQCountySDOHred = pd.concat([dfAHRQCountySDOHred2, dfAHRQCountySDOHsel], axis=1)
dfAHRQCountySDOHnew = dfAHRQCountySDOHred

### Missing values
Remove columns missing more than 30%.  Then impute missing data using KNN imputation.   Data set has many dimensions (about 1000) and comparably low data (about 3000).
Remove sparse columns.  KNN imputation may not work well with this shape.
May need to review all dimensions.

In [88]:
# drop columns that are more than 30% null
dfAHRQCountySDOHnew = dfAHRQCountySDOHnew.dropna(axis = 1, thresh=len(dfAHRQCountySDOHnew)*.7)

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5, weights="uniform")
dfAHRQCountySDOHnew.iloc[:,3::] = imputer.fit_transform(dfAHRQCountySDOHnew.iloc[:,3::])

### Load function

In [ ]:
# remove formatting issues by converting YEAR to int
dfAHRQCountySDOHnew["YEAR"] = pd.to_numeric(dfAHRQCountySDOHnew["YEAR"])
def out_AHRQCountySDOH():
   return dfAHRQCountySDOHnew